In [1]:
import pandas as pd
import numpy as np
import copy
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras import layers
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel("Dataset.xlsx",header=None)
X = data.iloc[:,6:10].values
Y = data.iloc[:,5].values
Y = Y.reshape(Y.shape[0],1)

In [3]:
train_df=pd.DataFrame(X)
train_df[4]=Y
train_df

,0,1,2,3,4
0,140,60,550,10,-10.10720
1,140,60,550,15,-10.41390
2,140,60,550,20,-12.26600
3,140,60,650,10,-12.27890
4,140,60,650,15,-11.95900
...,...,...,...,...,...
76,160,70,650,15,-11.17270
77,160,70,650,20,-9.31966
78,160,70,750,10,-10.29380
79,160,70,750,15,-11.88930


In [4]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train = scaler.fit_transform(train_df)

# Print out the adjustment that the scaler applied to the data
print("Note: Median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.min_[4], scaler.scale_[4]))
multiplied_by = scaler.min_[4]
added = scaler.scale_[4]

scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
scaled_train_df

Note: Median values were scaled by multiplying by 2.0609858632 and adding 0.139664


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.649370
1,0.0,0.0,0.0,0.5,0.606535
2,0.0,0.0,0.0,1.0,0.347862
3,0.0,0.0,0.5,0.0,0.346060
4,0.0,0.0,0.5,0.5,0.390739
...,...,...,...,...,...
76,1.0,1.0,0.5,0.5,0.500557
77,1.0,1.0,0.5,1.0,0.759361
78,1.0,1.0,1.0,0.0,0.623308
79,1.0,1.0,1.0,0.5,0.400474


In [6]:
X = scaled_train_df[scaled_train_df.columns[:4]].values
Y = scaled_train_df[scaled_train_df.columns[4]].values

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state = 0)

In [8]:
model = Sequential()
model.add(Dense(5, input_dim=4, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
op=Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(
    X_train,
    Y_train,
    epochs=200,
    shuffle=True,
    verbose=1
)

Epoch 1/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0733 - accuracy: 0.0179
Epoch 2/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0720 - accuracy: 0.0179
Epoch 3/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0711 - accuracy: 0.0179
Epoch 4/200
2/2 [==============================] - 0s 9ms/step - loss: 0.0699 - accuracy: 0.0179
Epoch 5/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0689 - accuracy: 0.0179
Epoch 6/200
2/2 [==============================] - 0s 10ms/step - loss: 0.0680 - accuracy: 0.0179
Epoch 7/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0669 - accuracy: 0.0179
Epoch 8/200
2/2 [==============================] - 0s 18ms/step - loss: 0.0661 - accuracy: 0.0357
Epoch 9/200
2/2 [==============================] - 0s 22ms/step - loss: 0.0652 - accuracy: 0.0357
Epoch 10/200
2/2 [==============================] - 0s 14ms/step - loss: 0.0643 - accuracy: 0.0357
Epoch 11/200
2/2 [=====

In [6]:
regr = make_pipeline(StandardScaler(), LinearRegression())
regr.fit(X_train,Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [7]:
regr.score(X_test,Y_test)

0.06617762111228087

In [8]:
Y_pred = regr.predict(X_test).reshape(Y_test.shape[0],1)

In [9]:
np.sum((Y_pred - Y_test)/Y_test)

0.8882887862648289

In [10]:
import pickle
pickle.dump(regr, open("Linear_Model.sav", 'wb'))